In [1]:
from glob import glob
from os import path
import seaborn as sns
import json
import os
import numpy as np
import pandas as pd
import re
from os.path import basename, splitext
import time
import pickle

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval_w_baseline

In [2]:
city = 'frank'

# fixed params
RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

refresh = False

In [3]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()

#Subset to frankfurt
list_city_dt_subset = []

for city_db in list_city_dt:
    if (city_db.find(city) > -1):
        list_city_dt_subset.append(city_db)

print(list_city_dt_subset)

['berlin_db=0.50', 'berlin_db=1.00', 'berlin_db=2.00', 'berlin_db=4.00', 'frankfurt_db=0.50', 'frankfurt_db=1.00', 'frankfurt_db=2.00', 'frankfurt_db=4.00', 'lyon_db=0.50', 'lyon_db=1.00', 'lyon_db=2.00', 'lyon_db=4.00', 'paris_db=0.50', 'paris_db=1.00', 'paris_db=2.00', 'paris_db=4.00']
16


In [6]:
for base_file in list_city_dt_subset:

    #create inputs to run a trial workforce size
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"

    expand_workforce_to_regions=True
    workforce_dict = {}

    for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    # for OC in [1.2]:
        for model in ['fixed', 'flex', 'partflex']:
        # for model in ['fixed']:
            if model == 'partflex':
                for max_n_shifts in range(2,5):

                    results_file = f"../raw_results/{base_file}_OC={OC}_model={model}_max_n_shifts={max_n_shifts}.pkl"

                    if not os.path.exists(results_file) or refresh:
                        start_time = time.time()
                        dict_baseline, dict_results = \
                            run_roster_solver_objval_w_baseline(model, instance_file_weekday, 
                                                                shift_file_weekday, instance_file_weekend, 
                                                                shift_file_weekend, workforce_dict, 
                                                                OC, RM, GM, h_min, h_max, max_n_diff, 
                                                                max_n_shifts, expand_workforce_to_regions=True)
                        
                        dict_results['baseline'] = dict_baseline
                        end_time = time.time()
                        run_time = (end_time - start_time)
                        #print(f'Base: {base_file}, OC: {OC}, model: {model}-{max_n_shifts}, run_time: {run_time}')
                        
                        # Save results
                        with open(results_file, 'wb') as file:
                            pickle.dump(dict_results, file)
                        print(f'instance_results saved to {results_file}')
            else:
                
                results_file = f"../raw_results/{base_file}_OC={OC}_model={model}.pkl"

                if not os.path.exists(results_file) or refresh:
                    start_time = time.time()
                    dict_baseline, dict_results = \
                        run_roster_solver_objval_w_baseline(model, instance_file_weekday, 
                                                            shift_file_weekday, instance_file_weekend, 
                                                            shift_file_weekend, workforce_dict, 
                                                            OC, RM, GM, h_min, h_max, max_n_diff, 
                                                            expand_workforce_to_regions=True)
                    
                    dict_results['baseline'] = dict_baseline
                    end_time = time.time()
                    run_time = (end_time - start_time)

                    # Save results
                    with open(results_file, 'wb') as file:
                        pickle.dump(dict_results, file)
                    print(f'instance_results saved to {results_file}')


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-12
instance_results saved to ../raw_results/frankfurt_db=0.50_OC=1.2_model=fixed.pkl
instance_results saved to ../raw_results/frankfurt_db=0.50_OC=1.2_model=flex.pkl
Base: frankfurt_db=0.50, OC: 1.2, model: partflex-2, run_time: 13.648569107055664
instance_results saved to ../raw_results/frankfurt_db=0.50_OC=1.2_model=partflex_max_n_shifts=2.pkl
Base: frankfurt_db=0.50, OC: 1.2, model: partflex-3, run_time: 16.72729516029358
instance_results saved to ../raw_results/frankfurt_db=0.50_OC=1.2_model=partflex_max_n_shifts=3.pkl
Base: frankfurt_db=0.50, OC: 1.2, model: partflex-4, run_time: 17.101038932800293
instance_results saved to ../raw_results/frankfurt_db=0.50_OC=1.2_model=partflex_max_n_shifts=4.pkl
instance_results saved to ../raw_results/frankfurt_db=0.50_OC=1.5_model=fixed.pkl
instance_results saved to ../raw_results/frankfurt_db=0.50_OC=1.5_model=flex.pkl
Base: frankfurt_db=0.50, OC: 1.5, mode